# Install and Import all the necessary packages and libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist, mnist, cifar10, cifar100
import numpy as np
import time
import math
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras import backend as K
import matplotlib.pyplot as plt
import pandas as pd
import gc
import multiprocessing
from multiprocessing import Process, Queue

# Definition of programmer's parameters (Controller's block)

In [2]:
#@markdown ##Dataset
topic_prefix = "cifar10" # @param ["cifar10", "cifar100", "mnist", "fashion_mnist"] {type:"string"}
if topic_prefix == "cifar10":
  dataset = cifar10.load_data()
elif topic_prefix == "cifar100":
  dataset = cifar100.load_data()
elif topic_prefix == "mnist":
  dataset = mnist.load_data()
elif topic_prefix == "fashion_mnist":
  dataset = fashion_mnist.load_data()

#@markdown ##Synopses-based Training Optimization Configuration
# Percentage of the dataset that will be used for training
sample_size_low = 0.05 #@param {type:"slider", min:0, max:0.99, step:0.05}
sample_size_step = 0.1 #@param {type:"number"}
sample_size_high = 1 #@param {type:"slider", min:0, max:1, step:0.05}

# Percentage of the dataset that will be used for testing
perc_test = 1 #@param {type:"number"}

# The ID of the method that will be employed during sampling
# 0: Simple reservoir sampling in our initial training dataset
# 1: Reservoir sampling in each class based on the number of samples (per class)
sampling_method_id = 1 #@param {type:"integer"}

#@markdown ##NN Architecture Configuration

total_num_of_layers = 5 #@param {type:"integer"}

set_of_layers = ['conv', 'pool', 'dense'] #, 'drop']

# Number of epochs for the training process
# One Epoch is when an ENTIRE (training) dataset is passed forward and backward through the neural network only once.
# NOTE: An epoch is comprised of one or more batches.
num_of_epochs_low = 1 #@param {type:"slider", min:1, max:30, step:1}
num_of_epochs_step = 4 #@param {type:"integer"}
num_of_epochs_high = 20 #@param {type:"slider", min:1, max:30, step:1}

# The value for the learning rate for the training process (it is inserted to the Adam optimizer)
# During the optimization, the algorithm needs to take a series of tiny steps to descend the error mountain in order to minimize the error.
# The direction of the step is determined by the gradient, while the step size is determined by the learning rate.
lr_low = 1e-3 #@param {type:"number"}
lr_high = 1e-3 #@param {type:"number"}
lr_list = np.geomspace(lr_low, lr_high, num = int(np.log10(lr_high) - np.log10(lr_low)) + 1).tolist()

# The size of the batch for the training process
# Total number of training examples present in a single batch.
# or, number of samples processed before the model is updated.
size_of_batch_low = 64 #@param {type:"integer"}
size_of_batch_step = 16 #@param {type:"integer"}
size_of_batch_high = 64 #@param {type:"integer"}

#@markdown ##Tradeoff score Configuration
# Maximum training time above which we cancel the experiment (in seconds)
# It is used in the score (accuracy--speed tradeoff) formula
theta_parameter = 10 #@param {type:"number"}

# Weight for the accuracy of the model. Max value: 0.99
# There is also the weight of training speed of the model (1 - lamda_acc)
# It is used in the score (accuracy--speed tradeoff) formula
lamda_acc = 0.5 #@param {type:"number"}


#@markdown ##Other Configuration(s)

CONV_NEURONS_CONST = 32 #@param {type:"integer"}
CONV_NEURONS_BOUND = 256 #@param {type:"integer"}
DENSE_NEURONS_CONST = 128 #@param {type:"integer"}
DENSE_NEURONS_BOUND = 32 #@param {type:"integer"}

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 15s 0us/step


# Load our image dataset

In [3]:
(train_images_all, train_labels_all), (test_images_all, test_labels_all) = dataset

train_images_all = np.vstack((train_images_all, train_images_all))
train_labels_all = np.concatenate((train_labels_all, train_labels_all))

# Get unique labels in our training dataset
unique_class_labels = np.unique(train_labels_all)

# Normalize pixel values in the trainning and the test datasets to be between 0 and 1
# TODO: parameter of normalization minmax scalar
train_images_all, test_images_all = train_images_all / 255.0, test_images_all / 255.0

if len(train_images_all.shape) == 3:
  # Make sure images have shape (28, 28, 1)
  train_images_all = np.expand_dims(train_images_all, -1)
  test_images_all = np.expand_dims(test_images_all, -1)
  print("x_train shape:", train_images_all.shape)
  print(train_images_all.shape[0], "train samples")
  print(test_images_all.shape[0], "test samples")

# Synopsis block of code

Optional: Check that our sampling method works appropriately

In [4]:
# A function that prints the occurence of each class in a list
def print_times_per_label(lst, labels_all):
  # Get unique labels in our training dataset
  unique_labels = np.unique(labels_all)
  for i in range(0, len(unique_labels)):
    print("Class", unique_labels[i], "has", lst.count(i), "samples in our dataset...")

**Reservoir Sampling Function**:
*Randomized algorithms for randomly choosing k samples from a list of n items, where n is either a very large or unknown number. Typically n is large enough that the list doesn’t fit into main memory.*

Algorithm implemented:


1.   Create an array reservoir[0,...,k-1] and copy first k items of stream[ ] to it.
2.   Now one by one consider all items from (k+1)th item to nth item.


> Steps

*   Generate a random number, denoted as j, from 0 to i where i is the index of the current item in stream[].
*   If j is in range 0 to k-1, replace reservoir[j] with stream[i]

In [5]:
# Select k items from a stream of items-data
import random

# A function to randomly select k items from stream[0..n-1].
def reservoir_sampling(stream, n, k):
  i = 0     # index for elements in stream[]

  # reservoir[] is the output array.
  # Initialize it with first k elements from stream[]
  reservoir = [0] * k

  for i in range(k):
    reservoir[i] = stream[i]

  # Iterate from the (k+1)th element to Nth element
  while(i < n):
    # Pick a random index from 0 to i.
    j = random.randrange(i+1)

    # If the randomly picked
    # index is smaller than k,
    # then replace the element
    # present at the index
    # with new element from stream
    if(j < k):
      reservoir[j] = stream[i]
    i+=1

  return reservoir

In [6]:
# A function that finds the size of each reservoir for every class depending on its occurence in the initial dataset
# and returns the unique labels that exist in our dataset along with the corresponding percentage
def reservoir_size_per_class(init_labels):

  # Get unique labels and their counts (how many times they appear) in our training dataset
  unique_labels, counts = np.unique(init_labels, return_counts = True)

  # Transform to list
  unique_labels_lst = unique_labels.tolist()
  counts_lst = counts.tolist()

  perc_per_class = []
  for i in range(len(unique_labels_lst)):
    perc_per_class.append(counts_lst[i]/len(init_labels))

  # print(perc_per_class)

  return perc_per_class, unique_labels_lst

"Pre-Processing Part (or Filtering)": Call the sampling mehod. Get samples from the training and the testing datasets.

In [7]:
def sampling_method(sampling_method_id, train_images_all, train_labels_all, sample_size, test_images_all, test_labels_all, perc_test):
  print("Percentage of filtering in our training dataset was set:")
  print(sample_size)
  if sampling_method_id == 0:
    # Simple reservoir sampling over the whole training dataset
    # Total size of the stream (or training dataset)
    n_train = len(train_images_all)

    # Number of samples that will be drawn
    k_train = int(n_train * sample_size)

    # Use the indexes of dataset in order to decide which samples will be drawn
    idx_tmp_train_list = list(range(0, n_train))

    # Find the indexes in order to construct the dataset that will be used during the training process
    idx_train = reservoir_sampling(idx_tmp_train_list, n_train, k_train)
  else:
    # Reservoir sampling in each class based on the number of samples (per class) that exist in the initial dataset
    # Find the size of each reservoir for every class depending on its occurence in the initial training dataset
    class_perc, unique_ids = reservoir_size_per_class(train_labels_all)

    # Stores the indexes (from all classes) in order to construct the dataset that will be used during the training process
    idx_train = []

    # Run for every single class the reservoir sampling seperately
    for i in range(0, len(unique_ids)):
      # Find the locations of each sample belonging to our class of interest
      tmp = np.where(train_labels_all == unique_ids[i])
      idx_of_class = tmp[0].tolist()

      # Run the reservoir sampling for the class of interest
      sampled_idx_of_class = reservoir_sampling(idx_of_class, len(idx_of_class), int(len(train_images_all) * sample_size * class_perc[i]))

      # Store the (sampled) samples from this class
      for j in range(0, len(sampled_idx_of_class)):
        idx_train.append(sampled_idx_of_class[j])

  # Store the corresponding images and labels from training dataset based on the sampled indexes
  train_images_lst = []
  for i in idx_train:
    train_images_lst.append(train_images_all[i])

  train_labels_lst = []
  for i in idx_train:
    train_labels_lst.append(train_labels_all[i])

  # Check the occurence of each class in the final training dataset
  print_times_per_label(train_labels_lst, train_labels_all)

  # Total size of the stream (or testing dataset)
  n_test = len(test_images_all)

  # Number of samples that will be drawn
  k_test = int(n_test * perc_test)

  # Use the indexes of dataset in order to decide which samples will be drawn
  idx_tmp_test_list = list(range(0, n_test))

  # Find the indexes in order to construct the dataset that will be used during the testing process
  idx_test = reservoir_sampling(idx_tmp_test_list, n_test, k_test)

  # Store the corresponding images and labels from testing dataset based on the sampled indexes
  test_images_lst = []
  for i in idx_test:
    test_images_lst.append(test_images_all[i])

  test_labels_lst = []
  for i in idx_test:
    test_labels_lst.append(test_labels_all[i])

  # Tranfsorm the lists that we stored our samples into arrays
  train_images = np.asarray(train_images_lst)
  train_labels = np.asarray(train_labels_lst)
  test_images = np.asarray(test_images_lst)
  test_labels = np.asarray(test_labels_lst)

  # Verify that the desired filtering was performed in both datasets
  print("Training dataset before sampling:")
  print(train_images_all.shape)
  print(train_labels_all.shape)
  print("Training dataset after sampling:")
  print(train_images.shape)
  print(train_labels.shape)

  print("Testing dataset before sampling:")
  print(test_images_all.shape)
  print(test_labels_all.shape)
  print("Testing dataset after sampling:")
  print(test_images.shape)
  print(test_labels.shape)

  return train_images, train_labels, test_images, test_labels

# CNN's architecture builder

In [8]:
def recreate_model(layers_lst, dataset_shape, CONV_NEURONS_CONST, DENSE_NEURONS_CONST, CONV_NEURONS_BOUND, DENSE_NEURONS_BOUND):

  # Initialize a sequential model
  model = models.Sequential()

  # Define the number of neurons for conv and dense layers
  conv_tmp2 = CONV_NEURONS_CONST
  dense_tmp2 = DENSE_NEURONS_CONST

  # Recreate the so-far-best-model
  for count, layer in enumerate(layers_lst):
    # First layer has to be a convolutional one
    if layer == 'conv' and count == 0:
      model.add(layers.Conv2D(int(conv_tmp2), (3, 3), activation='relu', input_shape = dataset_shape))
      conv_tmp2 = conv_tmp2 * 2
    # For the other layers
    else:
      if layer == 'conv':
        # Add a conv layer by doubling its neurons if they do not violate our user-defined bound
        if conv_tmp2 <= CONV_NEURONS_BOUND:
          model.add(layers.Conv2D(int(conv_tmp2), (3, 3), activation='relu'))
          conv_tmp2 = conv_tmp2 * 2
        else:
          model.add(layers.Conv2D(int(CONV_NEURONS_BOUND), (3, 3), activation='relu'))
          conv_tmp2 = CONV_NEURONS_BOUND
      elif layer == 'pool':
        # Add a pool layer
        model.add(layers.MaxPooling2D((2, 2), strides=(2,2), padding='same'))
      elif layer == 'drop':  # Dropout
        model.add(layers.Dropout(rate=0.5))
      else:
        # If the next to-be-added-layer is dense and no other dense layer has been added so far, then add a flatten layer first...
        if dense_tmp2 == DENSE_NEURONS_CONST:
          model.add(layers.Flatten())
        # Add a dense layer by reducing (* 0.5) its neurons if they do not violate our user-defined bound
        if dense_tmp2 >= DENSE_NEURONS_BOUND:
          model.add(layers.Dense(int(dense_tmp2), activation='relu'))
          dense_tmp2 = dense_tmp2 / 2
        else:
          model.add(layers.Dense(int(DENSE_NEURONS_BOUND), activation='relu'))
          dense_tmp2 = DENSE_NEURONS_BOUND

  return model, conv_tmp2, dense_tmp2

In [9]:
def my_evaluate_rest_phase(q, train_images, train_labels, test_images, test_labels, best_epochs, best_lr, best_batch, layer, best_layers, dataset_shape, CONV_NEURONS_CONST, DENSE_NEURONS_CONST, CONV_NEURONS_BOUND, DENSE_NEURONS_BOUND):

  error_flag = -1

  # Recreate the network that consist of the best layers that we found in each of the previous steps/iterations
  model, conv_tmp, dense_tmp = recreate_model(best_layers, dataset_shape, CONV_NEURONS_CONST, DENSE_NEURONS_CONST, CONV_NEURONS_BOUND, DENSE_NEURONS_BOUND)

  try:
    # If the to-be-added-layer is conv
    if layer == 'conv':
      # Add a conv layer by doubling its neurons if they do not violate our user-defined bound
      if conv_tmp <= CONV_NEURONS_BOUND:
        model.add(layers.Conv2D(int(conv_tmp), (3, 3), activation='relu'))
        conv_tmp = conv_tmp * 2
      else:
        model.add(layers.Conv2D(int(CONV_NEURONS_BOUND), (3, 3), activation='relu'))
    # If the to-be-added-layer is pool
    elif layer == 'pool':
      model.add(layers.MaxPooling2D((2, 2), strides=(2,2), padding='same'))
    # If the to-be-added-layer is dense
    elif layer == 'none':
      print("initial iterations of greedy have a fixed/predetermined architecture")
    elif layer == 'drop':  # Dropout
        model.add(layers.Dropout(rate=0.5))
    else:
      # If the next to-be-added-layer is dense and no other dense layer has been added so far, then add a flatten layer first...
      if not isinstance(model.layers[-1], tf.keras.layers.Dense):
        model.add(layers.Flatten())
      # Add a dense layer by reducing (* 0.5) its neurons if they do not violate our user-defined bound
      if dense_tmp >= DENSE_NEURONS_BOUND:
        model.add(layers.Dense(int(dense_tmp), activation='relu'))
        dense_tmp = dense_tmp / 2
      else:
        model.add(layers.Dense(int(DENSE_NEURONS_BOUND), activation='relu'))


    # Check if the last layer is a Dense layer
    last_layer = model.layers[-1]

    # Check if the last layer of the network is dense
    # If it is just add the last dense layer for the classification
    # Otherwise you should first add a flatten layer
    if isinstance(last_layer, tf.keras.layers.Dense):
        model.add(layers.Dense(len(unique_class_labels), activation='softmax'))
    else:
        model.add(layers.Flatten())
        model.add(layers.Dense(len(unique_class_labels), activation='softmax'))
  except ValueError:
    print("No valid input...:(")
    error_flag = 1

  if error_flag == -1:
    model.compile(optimizer=Adam(best_lr), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    model.summary()

    start = time.time()

    blackbox = model.fit(x=train_images,
                        y=train_labels,
                        epochs=best_epochs,
                        batch_size=best_batch
                        )
    stop = time.time()

    # Compute the training speed of this CNN architecture
    tr_time = stop - start

    # Compute the accuracy of our training model in the testing dataset
    test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

    # Compute the metric that captures the accuracy--speed tradeoff
    # Original formula
    # tradeOff_metric = lamda_acc * test_acc - (1 - lamda_acc) * math.tanh(tr_time/theta_parameter - 1)

    # Linear formula
    # tradeOff_metric = lamda_acc * test_acc + (1 - lamda_acc) * (1 - min(1, tr_time/theta_parameter))

    # Multiplicative formula
    # tradeOff_metric = test_acc ** lamda_acc * (1 - min(1, tr_time/theta_parameter)) ** (1 - lamda_acc)

    # Harmonic mean
    # tradeOff_metric = (2 * test_acc * (1 - min(1, tr_time/theta_parameter))) / (test_acc + 1 - min(1, tr_time/theta_parameter) + 1e-5)

    # Softmin
    # tradeOff_metric = 1 - np.log(1 + np.exp(-0.01 * test_acc) + np.exp(0.01 * min(1, tr_time/theta_parameter)))

    # Sigmoid
    # tradeOff_metric = (1 / (1 + np.exp(-test_acc))) * (1 - 1 / (1 + np.exp(-min(1, tr_time/theta_parameter))))

    # Piecewise
    # tradeOff_metric = test_acc if tr_time <= theta_parameter else test_acc * max(0, 1 - 0.5 * (tr_time - theta_parameter))

    # F1 like
    # alpha = 2
    # tradeOff_metric = ((1 + alpha**2) * test_acc * min(1, tr_time/theta_parameter)) / (alpha ** 2 * test_acc + min(1, tr_time/theta_parameter))

    # Exp decay penalty
    # tradeOff_metric = test_acc * np.exp(-0.5 * min(1, tr_time/theta_parameter))

    # inverse additive penalty
    tradeOff_metric = test_acc / (1 + 0.5 * min(1, tr_time/theta_parameter))

    # min-based tradeoff
    # tradeOff_metric = min(test_acc, min(1, tr_time/theta_parameter))

    # product with complement
    # tradeOff_metric = test_acc * (1 - min(1, tr_time/theta_parameter))

    # square root compensation
    # tradeOff_metric = np.sqrt(test_acc) * np.sqrt(1 - min(1, tr_time/theta_parameter))

    # jacard style normalization
    # tradeOff_metric = test_acc * (1 - min(1, tr_time/theta_parameter)) / (test_acc + (1 - min(1, tr_time/theta_parameter)) - test_acc * (1 - min(1, tr_time/theta_parameter)))

    # log-sigmoid dominance
    # tradeOff_metric = 1 / (1 + np.exp(-0.5 * (test_acc - (1 - min(1, tr_time/theta_parameter)))))


    del model

    print("========================== EDW EINAI TO MULTI2222222222.......")
    print("Accuracy (on the testing dataset): {0:.2%}".format(test_acc))
    print(f"Training time: ", tr_time)
    print(tradeOff_metric)

    q.put([test_acc, tr_time, tradeOff_metric])
  else:
    q.put([0, 1000000000, 0])


In [10]:
start_program = time.time()

# Get the shape of the input dataset
dataset_shape = train_images_all.shape[1:]

# Store the best type of layer of each step
default_layers = ['conv', 'pool', 'conv', 'pool', 'dense']

# Store the corresponding score. So the first position represents the score of the CNN that contains only the layer in the
# first position of best_layers list, the second the score of the CNN that contains the layers in the
# first and second position, etc.
best_scores = []
best_accuracies = []
best_times = []

# The best number of epochs, sample rate, learning rate and batch size
# This two quantities are computed during the first step/iteration, we keep the best ones and proceed to the
# next iterations with this numbers fixed
best_epochs = -1
best_sample_size = -1
best_lr = 1e-3
best_batch = 64

# Loop for the number of layers that user indicated
for step in range(1, 2 + total_num_of_layers + 1):
  # Store for each step/iteration the so far best score
  best_found = -1
  best_acc = -1
  best_t = 100000000000

  # If this is the first step/iteration
  if step == 1:

    # The first layer in this set of datasets has to be a convolutional one!
    # So our "search space" all combinations of epochs and sampling rates based on user's input
    for sample_size in np.arange(sample_size_low, sample_size_high + 0.01, sample_size_step):
      print()
      print("Sampling raaaate")
      print(sample_size)
      print()
      # Perform the sampling
      train_images, train_labels, test_images, test_labels = sampling_method(sampling_method_id, train_images_all, train_labels_all, sample_size, test_images_all, test_labels_all, perc_test)
      q = Queue()
      process_eval = multiprocessing.Process(target=my_evaluate_rest_phase, args=(q, train_images, train_labels, test_images, test_labels, 20, best_lr, best_batch, 'none', default_layers, dataset_shape, CONV_NEURONS_CONST, DENSE_NEURONS_CONST, CONV_NEURONS_BOUND, DENSE_NEURONS_BOUND))
      process_eval.start()
      test_acc, tr_time, tradeOff_metric = q.get()
      process_eval.join()

      # Print the results.
      print()
      print("========================== EDW EINAI TO CORE.......")
      print("Accuracy (on the testing dataset): {0:.2%}".format(test_acc))
      print(f"Training time: ", tr_time)
      print(tradeOff_metric)
      print()

      # Delete the dfs.
      del train_images
      del train_labels
      del test_images
      del test_labels

      # Clear the Keras session, otherwise it will keep adding new
      # models to the same TensorFlow graph each time we create
      # a model with a different set of hyper-parameters.
      K.clear_session()
      tf.compat.v1.reset_default_graph()

      # Check if this was the best score for this iteration
      if tradeOff_metric > best_found:
        best_found = tradeOff_metric
        best_acc = test_acc
        best_t = tr_time
        best_epochs = 20
        best_sample_size = sample_size
        best_lr = best_lr
        best_batch = best_batch

  # If this is not the first step/iteration
  elif step == 2:
      for epochs in range(num_of_epochs_low, num_of_epochs_high + 1, num_of_epochs_step):
        print()
        print("Epooochs")
        print(epochs)
        print()
        # Perform the sampling
        train_images, train_labels, test_images, test_labels = sampling_method(sampling_method_id, train_images_all, train_labels_all, best_sample_size, test_images_all, test_labels_all, perc_test)

        q = Queue()
        process_eval = multiprocessing.Process(target=my_evaluate_rest_phase, args=(q, train_images, train_labels, test_images, test_labels, epochs, best_lr, best_batch, 'none', default_layers, dataset_shape, CONV_NEURONS_CONST, DENSE_NEURONS_CONST, CONV_NEURONS_BOUND, DENSE_NEURONS_BOUND))
        process_eval.start()
        test_acc, tr_time, tradeOff_metric = q.get()
        process_eval.join()

        # Print the results.
        print()
        print("Accuracy (on the testing dataset): {0:.2%}".format(test_acc))
        print(f"Training time: ", tr_time)
        print(tradeOff_metric)
        print()

        # Delete the Keras model with these hyper-parameters from memory.
        K.clear_session()
        gc.collect()
        del train_images
        del train_labels
        del test_images
        del test_labels

        # Clear the Keras session, otherwise it will keep adding new
        # models to the same TensorFlow graph each time we create
        # a model with a different set of hyper-parameters.
        # K.clear_session()
        tf.compat.v1.reset_default_graph()

        # Check if this was the best score for this iteration
        if tradeOff_metric > best_found:
          best_found = tradeOff_metric
          best_acc = test_acc
          best_t = tr_time
          best_epochs = epochs
  else:
    if step == 3:
      best_layers = []
      best_layers.append('conv')

      # Perform the sampling
      train_images, train_labels, test_images, test_labels = sampling_method(sampling_method_id, train_images_all, train_labels_all, best_sample_size, test_images_all, test_labels_all, perc_test)
      q = Queue()
      process_eval = multiprocessing.Process(target=my_evaluate_rest_phase, args=(q, train_images, train_labels, test_images, test_labels, best_epochs, best_lr, best_batch, 'none', best_layers, dataset_shape, CONV_NEURONS_CONST, DENSE_NEURONS_CONST, CONV_NEURONS_BOUND, DENSE_NEURONS_BOUND))
      process_eval.start()
      test_acc, tr_time, tradeOff_metric = q.get()
      process_eval.join()

      # Print the results.
      print()
      print("========================== EDW EINAI TO CORE.......")
      print("Accuracy (on the testing dataset): {0:.2%}".format(test_acc))
      print(f"Training time: ", tr_time)
      print(tradeOff_metric)
      print()

      # Delete the dfs.
      del train_images
      del train_labels
      del test_images
      del test_labels

      # Clear the Keras session, otherwise it will keep adding new
      # models to the same TensorFlow graph each time we create
      # a model with a different set of hyper-parameters.
      K.clear_session()
      tf.compat.v1.reset_default_graph()

      best_scores.append(tradeOff_metric)
      best_accuracies.append(test_acc)
      best_times.append(tr_time)


    else:
      if best_layers[-1] == 'dense':
        best_layers.append('dense')
        # Perform the sampling
        train_images, train_labels, test_images, test_labels = sampling_method(sampling_method_id, train_images_all, train_labels_all, best_sample_size, test_images_all, test_labels_all, perc_test)
        q = Queue()
        process_eval = multiprocessing.Process(target=my_evaluate_rest_phase, args=(q, train_images, train_labels, test_images, test_labels, best_epochs, best_lr, best_batch, 'none', best_layers, dataset_shape, CONV_NEURONS_CONST, DENSE_NEURONS_CONST, CONV_NEURONS_BOUND, DENSE_NEURONS_BOUND))
        process_eval.start()
        test_acc, tr_time, tradeOff_metric = q.get()
        process_eval.join()

        # Print the results.
        print()
        print("========================== EDW EINAI TO CORE.......")
        print("Accuracy (on the testing dataset): {0:.2%}".format(test_acc))
        print(f"Training time: ", tr_time)
        print(tradeOff_metric)
        print()

        # Delete the dfs.
        del train_images
        del train_labels
        del test_images
        del test_labels

        # Clear the Keras session, otherwise it will keep adding new
        # models to the same TensorFlow graph each time we create
        # a model with a different set of hyper-parameters.
        K.clear_session()
        tf.compat.v1.reset_default_graph()

        best_scores.append(tradeOff_metric)
        best_accuracies.append(test_acc)
        best_times.append(tr_time)

      else:
        max_tradeoff = -1
        max_acc = -1
        max_t = -1
        max_layer = '-1'
        for i in range(0, len(set_of_layers)):
          print("Searching layer type for this layer")
          # Perform the sampling
          train_images, train_labels, test_images, test_labels = sampling_method(sampling_method_id, train_images_all, train_labels_all, best_sample_size, test_images_all, test_labels_all, perc_test)
          q = Queue()
          process_eval = multiprocessing.Process(target=my_evaluate_rest_phase, args=(q, train_images, train_labels, test_images, test_labels, best_epochs, best_lr, best_batch, set_of_layers[i], best_layers, dataset_shape, CONV_NEURONS_CONST, DENSE_NEURONS_CONST, CONV_NEURONS_BOUND, DENSE_NEURONS_BOUND))
          process_eval.start()
          test_acc, tr_time, tradeOff_metric = q.get()
          process_eval.join()

          # Print the results.
          print()
          print("========================== EDW EINAI TO CORE.......")
          print("Accuracy (on the testing dataset): {0:.2%}".format(test_acc))
          print(f"Training time: ", tr_time)
          print(tradeOff_metric)
          print()

          # Delete the dfs.
          del train_images
          del train_labels
          del test_images
          del test_labels

          # Clear the Keras session, otherwise it will keep adding new
          # models to the same TensorFlow graph each time we create
          # a model with a different set of hyper-parameters.
          K.clear_session()
          tf.compat.v1.reset_default_graph()

          if tradeOff_metric > max_tradeoff:
            max_tradeoff = tradeOff_metric
            max_acc = test_acc
            max_t = tr_time
            max_layer = set_of_layers[i]

        best_layers.append(max_layer)
        best_accuracies.append(max_acc)
        best_times.append(max_t)
        best_scores.append(max_tradeoff)



Sampling raaaate
0.05

Percentage of filtering in our training dataset was set:
0.05
Class 0 has 500 samples in our dataset...
Class 1 has 500 samples in our dataset...
Class 2 has 500 samples in our dataset...
Class 3 has 500 samples in our dataset...
Class 4 has 500 samples in our dataset...
Class 5 has 500 samples in our dataset...
Class 6 has 500 samples in our dataset...
Class 7 has 500 samples in our dataset...
Class 8 has 500 samples in our dataset...
Class 9 has 500 samples in our dataset...
Training dataset before sampling:
(100000, 32, 32, 3)
(100000, 1)
Training dataset after sampling:
(5000, 32, 32, 3)
(5000, 1)
Testing dataset before sampling:
(10000, 32, 32, 3)
(10000, 1)
Testing dataset after sampling:
(10000, 32, 32, 3)
(10000, 1)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


initial iterations of greedy have a fixed/predetermined architecture


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       401,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

 Trainable params: 2,077,130 (7.92 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/backend/tensorflow/nn.py:717: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1485/1485 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.4516 - loss: 1.5114
Epoch 2/5
1485/1485 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.6929 - loss: 0.8692
Epoch 3/5
1485/1485 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7808 - loss: 0.6262
Epoch 4/5
1485/1485 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8516 - loss: 0.4334
Epoch 5/5
1485/1485 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9088 - loss: 0.2661
313/313 - 3s - 9ms/step - accuracy: 0.6938 - loss: 1.3562
========================== EDW EINAI TO MULTI2222222222.......
Accuracy (on the testing dataset): 69.38%
Training time:  30.512282371520996
0.46253331502278644

========================== EDW EINAI TO CORE.......
Accuracy (on the testing dataset): 69.38%
Training time:  30.512282371520996
0.46253331502278644



In [11]:
stop_program = time.time()

# Results

In [12]:
print("================= Results ===============")
print(f"The best number of epochs that greedy algorithm found is: {best_epochs}")
print(f"The best sampling rate that greedy algorithm found is: {best_sample_size}")
print(f"The best number of lr that greedy algorithm found is: {best_lr}")
print(f"The best batch size that greedy algorithm found is: {best_batch}")

================= Results ===============
The best number of epochs that greedy algorithm found is: 5
The best sampling rate that greedy algorithm found is: 0.9500000000000002
The best number of lr that greedy algorithm found is: 0.001
The best batch size that greedy algorithm found is: 64


In [13]:
print(best_scores)
print()
print(best_accuracies)
print()
print(best_times)

[0.3925333420435588, 0.4289333422978719, 0.4577999909718831, 0.4764000177383423, 0.4838000138600667]

[0.5888000130653381, 0.6434000134468079, 0.6866999864578247, 0.7146000266075134, 0.7257000207901001]

[23.817777156829834, 23.762807846069336, 25.748223543167114, 28.077932119369507, 28.276169061660767]


In [14]:
# Compute the time
program_time = stop_program - start_program
program_time

1184.8760414123535

In [15]:
best_layers

['conv', 'pool', 'conv', 'conv', 'pool']